## Setup and Import

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



## Verify utility function

In [2]:
from utils import *
from Problem import Problem

# Since we cannot use the already given functions to compute the cost (different behaviours), we created a universal one that can correctly
# compute the cost of all given paths. This section verify the function
P = Problem(20, density = 0.2, alpha = 1, beta = 1)
baseline_path = get_baseline_path(P) 
# baseline_path
epsilon = 1e-7
print(f"My custom cost function verified: {abs(compute_cost(P, baseline_path)-P.baseline()) < epsilon}")


My custom cost function verified: True


## Test solution

In [3]:
import importlib
import pandas as pd
from Problem import Problem
from utils import compute_cost, check_feasibility
import s339425
import time

importlib.reload(s339425)
from s339425 import solution

n_cities = [10, 50, 100, 1000]
alpha_values = [0.0, 1.0, 2.0]
beta_values = [0.5, 1, 2]
density_values = [0.2, 0.5, 1.0]
seed = 42 

configs = []
for size in n_cities:
    for density in density_values:
        for alpha in alpha_values:
            for beta in beta_values:
                configs.append([size, density, alpha, beta])

results = []

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
RESET = "\033[0m"

print(f"{'SIZE':<5} {'DENS':<5} {'ALPHA':<5} {'BETA':<5} | {'STATUS':<8} | {'BASELINE':>10} | {'MY COST':>10} | {'IMPROV %':>8} | {'VALID':<8} | {'FOUND IN s':>8}")
print("-" * 85)

for size, density, alpha, beta in configs:
    test_p = Problem(size, density=density, alpha=alpha, beta=beta, seed=seed)
    
    start = time.time()
    baseline_cost = test_p.baseline()

    start = time.time()
    my_path = solution(test_p)
    elapsed = time.time()-start

    start = time.time()
    my_cost = compute_cost(test_p, my_path)
    
    if baseline_cost > 0:
        improvement = (baseline_cost - my_cost) / baseline_cost * 100
    else:
        improvement = 0.0

    if improvement > 0.001: 
        status_icon = "✅ WIN"
        color = GREEN
    elif improvement < -0.001: 
        status_icon = "❌ FAIL"
        color = RED
    else:
        status_icon = "👌 OK "
        color = YELLOW

    valid = check_feasibility(test_p, my_path)

    if valid:
        valid_icon = "✅ VALID"
        color_2 = GREEN
    else:
        valid_icon = "❌ NOT VALID"
        color_2 = RED

    print(f"{size:<5} {density:<5} {alpha:<5} {beta:<5} | {color}{status_icon:<8}{RESET} | {baseline_cost:10.2f} | {my_cost:10.2f} | {color}{improvement:7.2f}%{RESET} | {color_2}{valid_icon:<8}{RESET}| {elapsed:.3f}")
    
    results.append({
        'Size': size,
        'Density': density,
        'Alpha': alpha,
        'Beta': beta,
        'Status': status_icon.strip(),
        'Baseline': round(baseline_cost, 2),
        'Solution': round(my_cost, 2),
        'Improv%': round(improvement, 2),
        "Elapsed": round(elapsed, 3)
    })

df = pd.DataFrame(results)
print("\n" + "="*85)
print("RIEPILOGO FINALE")
print("="*85)
print(df.to_string(index=False))
print("="*85)

df.to_csv('test_results.csv', index=False)

SIZE  DENS  ALPHA BETA  | STATUS   |   BASELINE |    MY COST | IMPROV % | VALID    | FOUND IN s
-------------------------------------------------------------------------------------
10    0.2   0.0   0.5   | ✅ WIN    |      16.76 |       4.61 |   72.48% | ✅ VALID | 0.003
10    0.2   0.0   1     | ✅ WIN    |      16.76 |       4.61 |   72.48% | ✅ VALID | 0.001
10    0.2   0.0   2     | ✅ WIN    |      16.76 |       4.61 |   72.48% | ✅ VALID | 0.001
10    0.2   1.0   0.5   | ✅ WIN    |     301.47 |     300.02 |    0.48% | ✅ VALID | 0.002
10    0.2   1.0   1     | ✅ WIN    |    4896.29 |    4888.50 |    0.16% | ✅ VALID | 0.002
10    0.2   1.0   2     | ✅ WIN    | 1835859.98 |   12118.43 |   99.34% | ✅ VALID | 0.008
10    0.2   2.0   0.5   | ✅ WIN    |     419.40 |     417.56 |    0.44% | ✅ VALID | 0.001
10    0.2   2.0   1     | ✅ WIN    |    9775.81 |    9768.03 |    0.08% | ✅ VALID | 0.001
10    0.2   2.0   2     | ✅ WIN    | 7343389.65 |   19409.26 |   99.74% | ✅ VALID | 0.007
10    0.